<h1>Content<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-preparation" data-toc-modified-id="Data-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data preparation</a></span></li><li><span><a href="#Problem-research" data-toc-modified-id="Problem-research-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Problem research</a></span></li><li><span><a href="#Dealing-with-imbalance" data-toc-modified-id="Dealing-with-imbalance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dealing with imbalance</a></span></li><li><span><a href="#Model-testing" data-toc-modified-id="Model-testing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Model testing</a></span></li></ul></div>

# Customer churn

Customers began to leave Beta-Bank. Every month. A little, but noticeable. Banking marketers figured it was cheaper to keep current customers than to attract new ones.

It is necessary to predict whether the client will leave the bank in the near future or not. Provided historical data on customer behavior and termination of agreements with the bank.

It is necessary to build a model with an extremely large *F1*-measure (above 0.59).

Additionally, it is necessary to measure *AUC-ROC* and compare its value with *F1*-measure.

Data source: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Data preparation

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import warnings
#bd = pd.read_csv('Churn.csv') #bd - tariff_data
bd = pd.read_csv('/datasets/Churn.csv')

In [3]:
bd.info()
bd.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Based on a preliminary analysis of the available data, there were gaps in the Tenure column that need to be filled in, as their share is about 9% - a significant number.

Let's move on to data processing.

In [4]:
bd = bd.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

First of all, the RowNumber, CustomerId and Surname columns were removed, which do not carry any meaningful information, and the RowNumber column actually duplicates the indexes.

In [5]:
tenure_dict = (
    bd
    .pivot_table(index=['Geography', 'Gender', 'Age','NumOfProducts'], 
                 values='Tenure', 
                 aggfunc='median')
)

In [6]:
def tenure_index(row):
    Geography = row ['Geography']
    Gender = row['Gender']
    Age = row['Age']
    NumOfProducts = row['NumOfProducts']
    tenure_index = tuple([Geography, Gender, Age, NumOfProducts])
    return tenure_index
bd['tenure_index'] = bd.apply(tenure_index, axis=1)

In [7]:
bd['Tenure'] = (
    bd['Tenure']
    .fillna(bd['tenure_index'].map(tenure_dict['Tenure']))
)
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9975 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
tenure_index       10000 non-null object
dtypes: float64(3), int64(6), object(3)
memory usage: 937.6+ KB


Tenure categorized by Geography, Gender, Age, and NumOfProducts to fill in the missing values found, calculating the median for each of the possible options. After filling, out of the initially discovered gaps in the amount of 909 pieces, 25 remained.

In [8]:
bd.dropna(subset = ['Tenure'], inplace = True)
bd.reset_index(drop=True, inplace=True)
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9975 entries, 0 to 9974
Data columns (total 12 columns):
CreditScore        9975 non-null int64
Geography          9975 non-null object
Gender             9975 non-null object
Age                9975 non-null int64
Tenure             9975 non-null float64
Balance            9975 non-null float64
NumOfProducts      9975 non-null int64
HasCrCard          9975 non-null int64
IsActiveMember     9975 non-null int64
EstimatedSalary    9975 non-null float64
Exited             9975 non-null int64
tenure_index       9975 non-null object
dtypes: float64(3), int64(6), object(3)
memory usage: 935.3+ KB


In [9]:
bd = bd.drop(['tenure_index'], axis = 1)
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9975 entries, 0 to 9974
Data columns (total 11 columns):
CreditScore        9975 non-null int64
Geography          9975 non-null object
Gender             9975 non-null object
Age                9975 non-null int64
Tenure             9975 non-null float64
Balance            9975 non-null float64
NumOfProducts      9975 non-null int64
HasCrCard          9975 non-null int64
IsActiveMember     9975 non-null int64
EstimatedSalary    9975 non-null float64
Exited             9975 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 857.4+ KB


To avoid duplication when applying categorization with fewer parameters, rows with gaps have been removed along with the no longer needed categorization index column.

Let's move on to feature transformation.

In [10]:
bd_ohe = pd.get_dummies(bd, drop_first=True)
bd_ohe.info()
bd_ohe.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9975 entries, 0 to 9974
Data columns (total 12 columns):
CreditScore          9975 non-null int64
Age                  9975 non-null int64
Tenure               9975 non-null float64
Balance              9975 non-null float64
NumOfProducts        9975 non-null int64
HasCrCard            9975 non-null int64
IsActiveMember       9975 non-null int64
EstimatedSalary      9975 non-null float64
Exited               9975 non-null int64
Geography_Germany    9975 non-null uint8
Geography_Spain      9975 non-null uint8
Gender_Male          9975 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 730.7 KB


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8.0,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7.0,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4.0,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4.0,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2.0,134603.88,1,1,1,71725.73,0,0,0,1


To convert the categorical features in the data into numerical ones, a direct coding technique was used using an argument to exclude falling into a dummy trap.

Let's move on to sampling.

In [11]:
bd_train, bd_valid_test = train_test_split(
    bd_ohe, 
    test_size=0.40, 
    random_state=12345, 
    stratify=bd_ohe['Exited']
)
bd_valid, bd_test = train_test_split(
    bd_valid_test,                                      
    test_size=0.50, 
    random_state=12345, 
    stratify=bd_valid_test['Exited']
)
features_train = bd_train.drop(['Exited'], axis=1)
target_train = bd_train['Exited']
features_valid = bd_valid.drop(['Exited'], axis=1)
target_valid = bd_valid['Exited']
features_test = bd_test.drop(['Exited'], axis=1)
target_test = bd_test['Exited']

To split the data into three samples (training, validation and test), the train_test_split method was applied in two stages.  At the first stage, the first part related to the training set was selected, at the second stage, the second part was divided into half to obtain a validation and test set. The overall ratio of all three samples was 3:1:1 or 60%:20%:20%.
Further, each of the samples was divided into features (features) and target feature. Based on the task set before us, the target feature is the fact that the client leaves. It is the 'Exited' column in the available data.

Let's move on to feature scaling.

In [12]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Data standardization was applied to scale the available numerical features in the columns CreditScore, Age, Tenure, Balance, NumOfProducts, and EstimatedSalary.

We turn to the study of models.

## Problem research

In [13]:
%%time

warnings.filterwarnings("ignore")

best_model_DTC = None
best_result_DTC = 0
best_crit_DTC = ''
best_depth_DTC = 0
for crit in ['gini', 'entropy']:
    for depth in range(1, 100):
        model_DTC = DecisionTreeClassifier(random_state=12345, criterion=crit, max_depth=depth)
        model_DTC.fit(features_train, target_train)
        predicted_valid = model_DTC.predict(features_valid)
        result_DTC = f1_score(target_valid, predicted_valid)
        probabilities_valid = model_DTC.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc_DTC = roc_auc_score(target_valid, probabilities_one_valid)
        if result_DTC > best_result_DTC:
            best_model_DTC = model_DTC
            best_result_DTC = result_DTC
            best_crit_DTC = crit
            best_depth_DTC = depth
            best_auc_roc_DTC = auc_roc_DTC

print("F1-мера наилучшей модели решающего дерева на валидационной выборке:", best_result_DTC)
print("criterion наилучшей модели решающего дерева:", best_crit_DTC)
print("max_depth наилучшей модели решающего дерева:", best_depth_DTC)
print("AUC-ROC наилучшей модели решающего дерева на валидационной выборке:", best_auc_roc_DTC)

F1-мера наилучшей модели решающего дерева на валидационной выборке: 0.5525525525525525
criterion наилучшей модели решающего дерева: entropy
max_depth наилучшей модели решающего дерева: 6
AUC-ROC наилучшей модели решающего дерева на валидационной выборке: 0.8177172140694153
CPU times: user 6.38 s, sys: 35.4 ms, total: 6.41 s
Wall time: 6.5 s


Since the choice for the model consists of two choices in the Exited column, the target feature is categorical with a binary classification.  For modeling in the case of binary classification, let's start with a decision tree model.  To improve the model, we will form a cycle with the choice of hyperparameters: criterion with the options 'gini' and 'entropy', and max_depth in the range from 1 to 100. Based on the results of the work, the model with hyperparameters criterion - entropy and max_depth - 6 turned out to be the best model in terms of the F1-measure.  The AUC-ROC for this model is round 0.82, which is better than that of the random model (0.5), but still far from 1. The F1-measure turned out to be below our threshold of 0.59.

In [14]:
%%time

best_model_RFC = None
best_result_RFC = 0
best_est_RFC = 0
best_crit_RFC = ''
best_depth_RFC = 0
for est in range(1, 50):
    for crit in ['gini', 'entropy']:
        for depth in range(1, 25):
                model_RFC = RandomForestClassifier(random_state=12345, n_estimators=est, criterion=crit, max_depth=depth)
                model_RFC.fit(features_train, target_train)
                predicted_valid = model_RFC.predict(features_valid)
                result_RFC = f1_score(target_valid, predicted_valid)
                probabilities_valid = model_RFC.predict_proba(features_valid)
                probabilities_one_valid = probabilities_valid[:, 1]
                auc_roc_RFC = roc_auc_score(target_valid, probabilities_one_valid)
                if result_RFC > best_result_RFC:
                    best_model_RFC = model_RFC
                    best_result_RFC = result_RFC
                    best_est_RFC = est
                    best_crit_RFC = crit
                    best_depth_RFC = depth
                    best_auc_roc_RFC = auc_roc_RFC

print("F1-мера наилучшей модели случайного леса на валидационной выборке:", best_result_RFC)
print("n_estimators наилучшей модели случайного леса:", best_est_RFC)
print("criterion наилучшей модели случайного леса:", best_crit_RFC)
print("max_depth наилучшей модели случайного леса:", best_depth_RFC)
print("AUC-ROC наилучшей модели случайного леса на валидационной выборке:", best_auc_roc_RFC)

F1-мера наилучшей модели случайного леса на валидационной выборке: 0.5611940298507462
n_estimators наилучшей модели случайного леса: 13
criterion наилучшей модели случайного леса: entropy
max_depth наилучшей модели случайного леса: 19
AUC-ROC наилучшей модели случайного леса на валидационной выборке: 0.8149110955819551
CPU times: user 6min 13s, sys: 3.35 s, total: 6min 17s
Wall time: 6min 24s


The second binary classification model used the random forest model. To improve the model, a cycle was formed with the choice of hyperparameters: n_estimators in the range from 1 to 50, criterion with options 'gini' and 'entropy', and max_depth in the range from 1 to 25. According to the results of the work, the model with hyperparameters n_estimators turned out to be the best -  13, criterion - entropy and max_depth - 19. The F1-measure for it was greater than that of the best decision tree model, but still less than the threshold set in front of us at 0.59.  The AUC-ROC for this model is round 0.82, the same as for the decision tree model, which is better than the random model (0.5).

In [15]:
%%time

best_model_LR = None
best_result_LR = 0
best_solver_LR = ''
for sol in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model_LR = LogisticRegression(random_state=12345, solver=sol)
    model_LR.fit(features_train, target_train)
    predicted_valid = model_LR.predict(features_valid)
    result_LR = f1_score(target_valid, predicted_valid)
    probabilities_valid = model_LR.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_LR = roc_auc_score(target_valid, probabilities_one_valid)
    if result_LR > best_result_LR:
        best_model_LR = model_LR
        best_result_LR = result_LR
        best_solver_LR = sol
        best_auc_roc_LR = auc_roc_LR

print("F1-мера наилучшей модели логистической регрессии на валидационной выборке:", best_result_LR)
print("solver наилучшей модели логистической регрессии:", best_solver_LR)
print("AUC-ROC наилучшей модели логистической регрессии на валидационной выборке:", best_auc_roc_LR)

F1-мера наилучшей модели логистической регрессии на валидационной выборке: 0.29411764705882354
solver наилучшей модели логистической регрессии: liblinear
AUC-ROC наилучшей модели логистической регрессии на валидационной выборке: 0.7559251494681264
CPU times: user 378 ms, sys: 73.6 ms, total: 451 ms
Wall time: 130 ms


The third binary classification model used a logistic regression model. To improve the model, a loop was formed with the choice of the solver hyperparameter with options 'newton-cg', 'lbfgs', 'liblinear', 'sag' and 'saga'.  According to the results of the work carried out, the model with the hyperparameter solver - liblinear turned out to be the best. The F1-measure for it was significantly less than the threshold of 0.59 set before us, and, accordingly, less than that of the best decision tree and random forest models. The AUC-ROC of this model is also less than that of the best decision tree and random forest models, but still higher than that of the random model.

We turn to the elimination of imbalances to improve the results obtained.

## Dealing with imbalance

In [16]:
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

print("Размер таблицы отрицательных признаков:", features_zeros.shape)
print("Размер таблицы положительных признаков:", features_ones.shape)
print("Размер таблицы отрицательных значений целевого признака:", target_zeros.shape)
print("Размер таблицы положительных значений целевого признака:", target_ones.shape)

Размер таблицы отрицательных признаков: (4770, 11)
Размер таблицы положительных признаков: (1215, 11)
Размер таблицы отрицательных значений целевого признака: (4770,)
Размер таблицы положительных значений целевого признака: (1215,)


There is a significant difference in the number of objects of positive and negative class. There are many more negative signs.

Let's apply two options for dealing with the detected imbalance, using the functions of random decrease and increase in samples RandomUnderSampler and RandomOverSampler.

In [17]:
oversample = RandomOverSampler(sampling_strategy='minority')
features_train_over, target_train_over = oversample.fit_resample(features_train, 
                                                                 target_train)

In [18]:
features_zeros_over = features_train_over[target_train_over == 0]
features_ones_over = features_train_over[target_train_over == 1]
target_zeros_over = target_train_over[target_train_over == 0]
target_ones_over = target_train_over[target_train_over == 1]

print("Размер таблицы отрицательных признаков после случайного увеличения:", 
      features_zeros_over.shape)
print("Размер таблицы положительных признаков после случайного увеличения:", 
      features_ones_over.shape)
print("Размер таблицы отрицательных значений целевого признака после случайного увеличения:", 
      target_zeros_over.shape)
print("Размер таблицы положительных значений целевого признака после случайного увеличения:", 
      target_ones_over.shape)

Размер таблицы отрицательных признаков после случайного увеличения: (4770, 11)
Размер таблицы положительных признаков после случайного увеличения: (4770, 11)
Размер таблицы отрицательных значений целевого признака после случайного увеличения: (4770,)
Размер таблицы положительных значений целевого признака после случайного увеличения: (4770,)


In [19]:
undersample = RandomUnderSampler(sampling_strategy='majority')
features_train_under, target_train_under = undersample.fit_resample(features_train, 
                                                                 target_train)

In [20]:
features_zeros_under = features_train_under[target_train_under == 0]
features_ones_under = features_train_under[target_train_under == 1]
target_zeros_under = target_train_under[target_train_under == 0]
target_ones_under = target_train_under[target_train_under == 1]

print("Размер таблицы отрицательных признаков после случайного уменьшения:", 
      features_zeros_under.shape)
print("Размер таблицы положительных признаков после случайного уменьшения:", 
      features_ones_under.shape)
print("Размер таблицы отрицательных значений целевого признака после случайного уменьшения:", 
      target_zeros_under.shape)
print("Размер таблицы положительных значений целевого признака после случайного уменьшения:", 
      target_ones_under.shape)

Размер таблицы отрицательных признаков после случайного уменьшения: (1215, 11)
Размер таблицы положительных признаков после случайного уменьшения: (1215, 11)
Размер таблицы отрицательных значений целевого признака после случайного уменьшения: (1215,)
Размер таблицы положительных значений целевого признака после случайного уменьшения: (1215,)


Based on the results, training samples were formed with two options for dealing with imbalance. Next, we will re-select the models, but using these samples.

In [21]:
%%time

warnings.filterwarnings("ignore")

best_model_DTC_over = None
best_result_DTC_over = 0
best_crit_DTC_over = ''
best_depth_DTC_over = 0
best_auc_roc_DTC_over = 0
for crit in ['gini', 'entropy']:
    for depth in range(1, 100):
        model_DTC_over = DecisionTreeClassifier(random_state=12345, criterion=crit, max_depth=depth)
        model_DTC_over.fit(features_train_over, target_train_over)
        predicted_valid = model_DTC_over.predict(features_valid)
        result_DTC_over = f1_score(target_valid, predicted_valid)
        probabilities_valid = model_DTC_over.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc_DTC_over = roc_auc_score(target_valid, probabilities_one_valid)
        if result_DTC_over > best_result_DTC_over:
            best_model_DTC_over = model_DTC_over
            best_result_DTC_over = result_DTC_over
            best_crit_DTC_over = crit
            best_depth_DTC_over = depth
            best_auc_roc_DTC_over = auc_roc_DTC_over

print("F1-мера наилучшей модели решающего дерева на валидационной выборке после случайного увеличения:", 
      best_result_DTC_over)
print("criterion наилучшей модели решающего дерева после случайного увеличения:", best_crit_DTC_over)
print("max_depth наилучшей модели решающего дерева после случайного увеличения:", best_depth_DTC_over)
print("AUC-ROC наилучшей модели решающего дерева после случайного увеличения на валидационной выборке:", 
      best_auc_roc_DTC_over)

F1-мера наилучшей модели решающего дерева на валидационной выборке после случайного увеличения: 0.5472727272727272
criterion наилучшей модели решающего дерева после случайного увеличения: gini
max_depth наилучшей модели решающего дерева после случайного увеличения: 5
AUC-ROC наилучшей модели решающего дерева после случайного увеличения на валидационной выборке: 0.8150415404922743
CPU times: user 7.87 s, sys: 64.8 ms, total: 7.93 s
Wall time: 8.02 s


To improve the model, we will form a cycle with the choice of hyperparameters: criterion with options 'gini' and 'entropy', and max_depth in the range from 1 to 100. According to the results of the work, the best model after randomly increasing the F1-measure turned out to be a model with hyperparameters criterion - entropy and  max_depth - 5. AUC-ROC for this model increased slightly and is rounded also 0.82, which is better than that of the random model (0.5), but still far from 1. F1-measure also increased and was rounded 0.56,  which is still less than the threshold of 0.59.

In [22]:
%%time

best_model_RFC_over = None
best_result_RFC_over = 0
best_est_RFC_over = 0
best_crit_RFC_over = ''
best_depth_RFC_over = 0
best_auc_roc_RFC_over = 0
for est in range(1, 50):
    for crit in ['gini', 'entropy']:
        for depth in range(1, 25):
                model_RFC_over = RandomForestClassifier(random_state=12345, n_estimators=est, criterion=crit, max_depth=depth)
                model_RFC_over.fit(features_train_over, target_train_over)
                predicted_valid = model_RFC_over.predict(features_valid)
                result_RFC_over = f1_score(target_valid, predicted_valid)
                probabilities_valid = model_RFC_over.predict_proba(features_valid)
                probabilities_one_valid = probabilities_valid[:, 1]
                auc_roc_RFC_over = roc_auc_score(target_valid, probabilities_one_valid)
                if result_RFC_over > best_result_RFC_over:
                    best_model_RFC_over = model_RFC_over
                    best_result_RFC_over = result_RFC_over
                    best_est_RFC_over = est
                    best_crit_RFC_over = crit
                    best_depth_RFC_over = depth
                    best_auc_roc_RFC_over = auc_roc_RFC_over

print("F1-мера наилучшей модели случайного леса на валидационной выборке после случайного увеличения:", 
      best_result_RFC_over)
print("n_estimators наилучшей модели случайного леса после случайного увеличения:", best_est_RFC_over)
print("criterion наилучшей модели случайного леса после случайного увеличения:", best_crit_RFC_over)
print("max_depth наилучшей модели случайного леса после случайного увеличения:", best_depth_RFC_over)
print("AUC-ROC наилучшей модели случайного леса после случайного увеличения на валидационной выборке:", 
      best_auc_roc_RFC_over)

F1-мера наилучшей модели случайного леса на валидационной выборке после случайного увеличения: 0.6079182630906769
n_estimators наилучшей модели случайного леса после случайного увеличения: 43
criterion наилучшей модели случайного леса после случайного увеличения: entropy
max_depth наилучшей модели случайного леса после случайного увеличения: 14
AUC-ROC наилучшей модели случайного леса после случайного увеличения на валидационной выборке: 0.8409255376970262
CPU times: user 7min 53s, sys: 2.7 s, total: 7min 55s
Wall time: 8min


To improve the random forest model, a cycle was formed with the choice of hyperparameters: n_estimators in the range from 1 to 50, criterion with options 'gini' and 'entropy', and max_depth in the range from 1 to 25. According to the results of the work, the best model after random increase turned out to be  the model with hyperparameters n_estimators - 16, criterion - entropy and max_depth - 11. The F1-measure for it increased significantly compared to the model without random increase and amounted to 0.62, which is more than the threshold set in front of us at 0.59. The AUC-ROC for this model is round 0.84, an improvement over the past and currently the highest of any model studied.

In [23]:
%%time

best_model_LR_over = None
best_result_LR_over = 0
best_solver_LR_over = ''
best_auc_roc_LR_over = 0
for sol in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model_LR_over = LogisticRegression(random_state=12345, solver=sol)
    model_LR_over.fit(features_train_over, target_train_over)
    predicted_valid = model_LR_over.predict(features_valid)
    result_LR_over = f1_score(target_valid, predicted_valid)
    probabilities_valid = model_LR_over.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_LR_over = roc_auc_score(target_valid, probabilities_one_valid)
    if result_LR_over > best_result_LR_over:
        best_model_LR_over = model_LR_over
        best_result_LR_over = result_LR_over
        best_solver_LR_over = sol
        best_auc_roc_LR_over = auc_roc_LR_over

print("F1-мера наилучшей модели логистической регрессии на валидационной выборке после случайного увеличения:", 
      best_result_LR_over)
print("solver наилучшей модели логистической регрессии после случайного увеличения:", best_solver_LR_over)
print("AUC-ROC наилучшей модели логистической регрессии на валидационной выборке после случайного увеличения:", 
      best_auc_roc_LR_over)

F1-мера наилучшей модели логистической регрессии на валидационной выборке после случайного увеличения: 0.495985727029438
solver наилучшей модели логистической регрессии после случайного увеличения: newton-cg
AUC-ROC наилучшей модели логистической регрессии на валидационной выборке после случайного увеличения: 0.7611802158552682
CPU times: user 512 ms, sys: 109 ms, total: 621 ms
Wall time: 179 ms


The best logistic regression model, as well as the decision tree and random forest models, showed better results after random increase than before random increase, but this did not help it rise from the last place in terms of indicators - they are still the same low and significantly worse.  from the best results of other models.

Next, we present a grouped selection of decision tree, random forest, and logistic regression models on samples with random reduction, and then draw a general conclusion after dealing with the imbalance.

In [24]:
%%time

warnings.filterwarnings("ignore")

best_model_DTC_under = None
best_result_DTC_under = 0
best_crit_DTC_under = ''
best_depth_DTC_under = 0
best_auc_roc_DTC_under = 0
for crit in ['gini', 'entropy']:
    for depth in range(1, 100):
        model_DTC_under = DecisionTreeClassifier(random_state=12345, criterion=crit, max_depth=depth)
        model_DTC_under.fit(features_train_under, target_train_under)
        predicted_valid = model_DTC_under.predict(features_valid)
        result_DTC_under = f1_score(target_valid, predicted_valid)
        probabilities_valid = model_DTC_under.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        auc_roc_DTC_under = roc_auc_score(target_valid, probabilities_one_valid)
        if result_DTC_under > best_result_DTC_under:
            best_model_DTC_under = model_DTC_under
            best_result_DTC_under = result_DTC_under
            best_crit_DTC_under = crit
            best_depth_DTC_under = depth
            best_auc_roc_DTC_under = auc_roc_DTC_under

print("F1-мера наилучшей модели решающего дерева на валидационной выборке после случайного уменьшения:", 
      best_result_DTC_under)
print("criterion наилучшей модели решающего дерева после случайного уменьшения:", best_crit_DTC_under)
print("max_depth наилучшей модели решающего дерева после случайного уменьшения:", best_depth_DTC_under)
print("AUC-ROC наилучшей модели решающего дерева на валидационной выборке после случайного уменьшения:", 
      best_auc_roc_DTC_under)

F1-мера наилучшей модели решающего дерева на валидационной выборке после случайного уменьшения: 0.5517948717948719
criterion наилучшей модели решающего дерева после случайного уменьшения: gini
max_depth наилучшей модели решающего дерева после случайного уменьшения: 6
AUC-ROC наилучшей модели решающего дерева на валидационной выборке после случайного уменьшения: 0.8161712865905739
CPU times: user 3.33 s, sys: 65.1 ms, total: 3.4 s
Wall time: 3.22 s


In [25]:
%%time

best_model_RFC_under = None
best_result_RFC_under = 0
best_est_RFC_under = 0
best_crit_RFC_under = ''
best_depth_RFC_under = 0
best_auc_roc_RFC_under = 0
for est in range(1, 50):
    for crit in ['gini', 'entropy']:
        for depth in range(1, 25):
                model_RFC_under = RandomForestClassifier(random_state=12345, n_estimators=est, criterion=crit, max_depth=depth)
                model_RFC_under.fit(features_train_under, target_train_under)
                predicted_valid = model_RFC_under.predict(features_valid)
                result_RFC_under = f1_score(target_valid, predicted_valid)
                probabilities_valid = model_RFC_under.predict_proba(features_valid)
                probabilities_one_valid = probabilities_valid[:, 1]
                auc_roc_RFC_under = roc_auc_score(target_valid, probabilities_one_valid)
                if result_RFC_under > best_result_RFC_under:
                    best_model_RFC_under = model_RFC_under
                    best_result_RFC_under = result_RFC_under
                    best_est_RFC_under = est
                    best_crit_RFC_under = crit
                    best_depth_RFC_under = depth
                    best_auc_roc_RFC_under = auc_roc_RFC_under

print("F1-мера наилучшей модели случайного леса на валидационной выборке после случайного уменьшения:", 
      best_result_RFC_under)
print("n_estimators наилучшей модели случайного леса после случайного уменьшения:", best_est_RFC_under)
print("criterion наилучшей модели случайного леса после случайного уменьшения:", best_crit_RFC_under)
print("max_depth наилучшей модели случайного леса после случайного уменьшения:", best_depth_RFC_under)
print("AUC-ROC наилучшей модели случайного леса на валидационной выборке после случайного уменьшения:", 
      best_auc_roc_RFC_under)

F1-мера наилучшей модели случайного леса на валидационной выборке после случайного уменьшения: 0.5816023738872403
n_estimators наилучшей модели случайного леса после случайного уменьшения: 34
criterion наилучшей модели случайного леса после случайного уменьшения: gini
max_depth наилучшей модели случайного леса после случайного уменьшения: 4
AUC-ROC наилучшей модели случайного леса на валидационной выборке после случайного уменьшения: 0.8371868933923442
CPU times: user 3min 29s, sys: 864 ms, total: 3min 29s
Wall time: 3min 30s


In [26]:
%%time

best_model_LR_under = None
best_result_LR_under = 0
best_solver_LR_under = ''
best_auc_roc_LR_under = 0
for sol in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model_LR_under = LogisticRegression(random_state=12345, solver=sol)
    model_LR_under.fit(features_train_under, target_train_under)
    predicted_valid = model_LR_under.predict(features_valid)
    result_LR_under = f1_score(target_valid, predicted_valid)
    probabilities_valid = model_LR_under.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_LR_under = roc_auc_score(target_valid, probabilities_one_valid)
    if result_LR_under > best_result_LR_under:
        best_model_LR_under = model_LR_under
        best_result_LR_under = result_LR_under
        best_solver_LR_under = sol
        best_auc_roc_LR_under = auc_roc_LR_under

print("F1-мера наилучшей модели логистической регрессии на валидационной выборке после случайного уменьшения:", 
      best_result_LR_under)
print("solver наилучшей модели логистической регрессии после случайного уменьшения:", best_solver_LR_under)
print("AUC-ROC наилучшей модели логистической регрессии на валидационной выборке после случайного уменьшения:", 
      best_auc_roc_LR_under)

F1-мера наилучшей модели логистической регрессии на валидационной выборке после случайного уменьшения: 0.4968944099378882
solver наилучшей модели логистической регрессии после случайного уменьшения: newton-cg
AUC-ROC наилучшей модели логистической регрессии на валидационной выборке после случайного уменьшения: 0.7602329373398556
CPU times: user 228 ms, sys: 61.5 ms, total: 289 ms
Wall time: 85.5 ms


After testing the prepared samples after random increase and decrease on the best decision tree, random forest and logistic regression models, mixed results were obtained.
The main thing that can be noted is that the use of training samples after a random decrease led to a drop in some of the indicators of the models, but at the same time, the indicators of the logistic regression model in this case increased, but did not exceed the required level of the F1-measure (0.59), and F1-measure (and only it without the AUC-ROC indicator) increased and reached 0.59.
In the case of sampling, after a random increase, the performance of all models increased, although in some places by a very small amount. The leader in all indicators, as before the application of the fight against imbalance, was the random forest model.
Due to the results obtained after adjusting the imbalance, further testing will be performed on the best random forest model with the following parameters: n_estimators - 16, criterion - entropy, max_depth - 11.

## Model testing

In [27]:
features_train_valid_over = features_train_over.append(features_valid)
target_train_valid_over = target_train_over.append(target_valid)

In [28]:
best_model_RFC_over.fit(features_train_valid_over, target_train_valid_over)
predicted_test_over = best_model_RFC_over.predict(features_test)
result_test_over = f1_score(target_test, predicted_test_over)
probabilities_test_over = best_model_RFC_over.predict_proba(features_test)
probabilities_one_test_over = probabilities_test_over[:, 1]
auc_roc_test_over = roc_auc_score(target_test, probabilities_one_test_over)
print("F1-мера наилучшей модели случайного леса на тестовой выборке после случайного увеличения:", 
      result_test_over)
print("AUC-ROC наилучшей модели случайного леса на тестовой выборке после случайного увеличения:", 
      auc_roc_test_over)

F1-мера наилучшей модели случайного леса на тестовой выборке после случайного увеличения: 0.6230769230769231
AUC-ROC наилучшей модели случайного леса на тестовой выборке после случайного увеличения: 0.8566953956052488


The best random tree model successfully passed the final testing with the improvement of previously obtained control parameters.

The results of our work have shown the importance of dealing with imbalances and a significant improvement in results after its application.

We transfer this model to further work on predicting the outflow of customers.